Preliminary

In [1]:
# Import

import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [2]:
# Download training data from open datasets.
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

In [3]:
# Pass the Dataset as an argument to DataLoader
# batch size of 64
batch_size = 64
# Create Data Loaders
train_dataloader = DataLoader(training_data, batch_size = batch_size)
test_dataloader = DataLoader(test_data, batch_size = batch_size)

# iterate over to return a batch of 64 features and labels
for (X, y) in test_dataloader:
  print(f"Shape of X [N, C, H, W]: {X.shape}")
  print(f"Shape of y: {y.shape} {y.dtype}")

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64
Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64
Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64
Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64
Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64
Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64
Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64
Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64
Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64
Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64
Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape o

In [4]:
# Use accelerator if available
device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
print(f"Using {device} device")

# Define the model by intialising a class
class NeuralNetwork(nn.Module):
    # initailising method
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(), 
            nn.Linear(512, 512),
            nn.ReLU(), 
            nn.Linear(512, 10)
        )
    # forward propagation
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits
    
model = NeuralNetwork().to(device)
# View the structure of the model you have specified
print(model)


Using cuda device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [6]:
# Optimize the actual parameters
# This requires specifying the loss function and the optimizer
# Cross Entropy for classifying images
loss_fn = nn.CrossEntropyLoss()
# We have opted to use an SGD optimizer with the corresponding learning rate
optimizer = torch.optim.SGD(model.parameters(), lr = 1e-3)

In [7]:
# Define a training loop
# this takes in arguments
# dataloader
# model
# loss_fn
# optimizer
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        
        # compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)
        # Backpropogate the loss
        loss.backward()
        optimizer.step()
        # zero out the gradient to avoid them accumulating
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f} [{current:>5d} / {size:>5d}]")



In [9]:
# Define a VALIDATION step 
# this is somewhat erroneously labelled as a TEST loop
# But this isn't true, because we are using this step for validation monitoring 
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    # with is like a try-catch
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            # really validation loss, but whatever...
            # .item() returns the dictionaries key-value pairs...
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    # same as test_loss = test_loss / num_batches
    test_loss /= num_batches
    correct /= size 
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")


In [10]:
# Conduct the training over serveal iterations (epochs)
# You will need to manually print the model's accuracy and loss at each epoch
epochs = 5
for t in range(epochs):
    print(f"Epoch {t + 1}\n ---------------------------")
    # call one trianing step
    train(train_dataloader, model, loss_fn, optimizer)
    # call one test/eval set
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
 ---------------------------
loss: 2.168386 [   64 / 60000]
loss: 2.162688 [ 6464 / 60000]
loss: 2.105719 [12864 / 60000]
loss: 2.102085 [19264 / 60000]
loss: 2.071215 [25664 / 60000]
loss: 2.005635 [32064 / 60000]
loss: 2.027830 [38464 / 60000]
loss: 1.949310 [44864 / 60000]
loss: 1.948059 [51264 / 60000]
loss: 1.873625 [57664 / 60000]
Test Error: 
 Accuracy: 55.0%, Avg loss: 1.875757 

Epoch 2
 ---------------------------
loss: 1.919101 [   64 / 60000]
loss: 1.887505 [ 6464 / 60000]
loss: 1.768767 [12864 / 60000]
loss: 1.788214 [19264 / 60000]
loss: 1.705302 [25664 / 60000]
loss: 1.648240 [32064 / 60000]
loss: 1.665112 [38464 / 60000]
loss: 1.565531 [44864 / 60000]
loss: 1.590053 [51264 / 60000]
loss: 1.486315 [57664 / 60000]
Test Error: 
 Accuracy: 59.7%, Avg loss: 1.502864 

Epoch 3
 ---------------------------
loss: 1.577037 [   64 / 60000]
loss: 1.539165 [ 6464 / 60000]
loss: 1.389199 [12864 / 60000]
loss: 1.449121 [19264 / 60000]
loss: 1.355939 [25664 / 60000]
loss: 1.33

In [11]:
# Saving the Model
# One such common way to save a model to is serialize the internal state dictionary
# i.e. save the model parameters
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


In [12]:
# Loading Models
model = NeuralNetwork().to(device)
model.load_state_dict(torch.load("model.pth", weights_only = True))

<All keys matched successfully>

In [17]:
test_data[0][1]

9

In [21]:
# Make predictions
# Load up classes first (10 in total)
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]
#
model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    x = x.to(device)
    pred = model(x)
    # torch.argmax returns the indices of the max value of all elements
    # what does the extra 0 in the brackets do?
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')


Predicted: "Ankle boot", Actual: "Ankle boot"
